# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [24]:
# Use this cell to write your code for Task 1
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv('house_sales.csv')

# Replace the missing data with nan
df['city'].replace('--', np.nan, inplace=True)

# Calculate the number of missing values in the 'city' column
missing_city = df['city'].isnull().sum()

print(missing_city)

73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [25]:
# Use this cell to write your code for Task 2

import pandas as pd

# Load the data
df = pd.read_csv("house_sales.csv")

# Handling missing values in the 'city' column
df['city'].replace('--', 'Unknown', inplace=True)


# Removing missing entries in the 'sale_price' column
df.dropna(subset=['sale_price'], inplace=True)
df = df[df['sale_price'] > 0]

# Handling missing values in the 'sale_date' column
df['sale_date'].fillna("2023-01-01", inplace=True)

# Cleaning the 'months_listed' column
mean_months_listed = df['months_listed'].mean()
df['months_listed'].fillna(mean_months_listed, inplace=True)
df['months_listed'] = df['months_listed'].round(1)

# Cleaning the 'bedrooms' column
mean_bedrooms = df['bedrooms'].mean()
df['bedrooms'].fillna(round(mean_bedrooms), inplace=True)

# Cleaning the 'house_type' column
df['house_type'].replace({'Det.': 'Detached', 'Semi': 'Semi-detached', 'Terr.': 'Terraced'}, inplace=True)
most_common_house_type = df['house_type'].mode()[0]
df['house_type'].fillna(most_common_house_type, inplace=True)

# Cleaning the 'area' column
df['area'] = df['area'].str.extract(r'(\d+\.\d+)').astype(float)


# Create the cleaned dataframe
clean_data = df.copy()

# Displaying the resulting DataFrame
print(clean_data)

      house_id        city  sale_price  ... bedrooms     house_type   area
0      1217792  Silvertown       55943  ...        2  Semi-detached  107.8
1      1900913  Silvertown      384677  ...        5       Detached  498.8
2      1174927   Riverford      281707  ...        6       Detached  542.5
3      1773666  Silvertown      373251  ...        6       Detached  528.4
4      1258487  Silvertown      328885  ...        5       Detached  477.1
...        ...         ...         ...  ...      ...            ...    ...
1495   1123892   Riverford      198661  ...        5       Detached  432.2
1496   1327295   Poppleton      358304  ...        6       Detached  599.8
1497   1058161   Riverford      176612  ...        4       Detached  359.1
1498   1822037    Teasdale      197827  ...        3       Detached  253.7
1499   1679272   Poppleton      162099  ...        3       Detached  268.7

[1500 rows x 8 columns]


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [26]:
# Use this cell to write your code for Task 3

df = pd.read_csv("house_sales.csv")

# Replacing the missing values in the 'bedrooms' column with the mean
mean_bedrooms = df['bedrooms'].mean()
df['bedrooms'].fillna(round(mean_bedrooms), inplace=True)

# Calculating average price and variance by number of bedrooms
price_by_rooms = df.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()
price_by_rooms.columns = ['bedrooms', 'avg_price', 'var_price']

# Rounding off the values to 1 decimal place
price_by_rooms = price_by_rooms.round(1)

# Displaying the resulting DataFrame
print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [27]:
# Use this cell to write your code for Task 4

# Load the dataset
train_data = pd.read_csv("train.csv")
validation_data = pd.read_csv("validation.csv")

# Use get_dummies for one hot encoding
train_data = pd.get_dummies(train_data, columns=["city", "house_type"], drop_first=True)
validation_data = pd.get_dummies(validation_data, columns=["city", "house_type"], drop_first=True)

# Convert to date
train_data["sale_date"] = pd.to_datetime(train_data["sale_date"])
validation_data["sale_date"] = pd.to_datetime(validation_data["sale_date"])

# To date
train_data["sale_year"] = train_data["sale_date"].dt.year
train_data["sale_month"] = train_data["sale_date"].dt.month
train_data["sale_day"] = train_data["sale_date"].dt.day

# To date
validation_data["sale_year"] = validation_data["sale_date"].dt.year
validation_data["sale_month"] = validation_data["sale_date"].dt.month
validation_data["sale_day"] = validation_data["sale_date"].dt.day

# Declare target variable
target_variable = "sale_price"

# Create the features and the target
X = train_data.drop(columns=[target_variable, "sale_date"])
y = train_data[target_variable]

# Split the dataset into testing and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Use LinReg for baseline model
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

# Use a validation set
X_validation = validation_data.drop(columns=["sale_date"])
predicted_prices = baseline_model.predict(X_validation)

base_result = pd.DataFrame({"house_id": validation_data["house_id"], "price": predicted_prices})

print(base_result)

     house_id          price
0     1331375  122123.371414
1     1630115  301093.968201
2     1645745  383868.093415
3     1336775  124987.007602
4     1888274  271631.042705
..        ...            ...
295   1986255  349514.854794
296   1896276  370161.455263
297   1758223  259358.691152
298   1752010  166744.151123
299   1651404  389046.798381

[300 rows x 2 columns]


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [28]:
# Use this cell to write your code for Task 5

# For comparison, use an RF Model
comparison_model = RandomForestRegressor(random_state=34)
comparison_model.fit(X_train, y_train)

X_validation = validation_data.drop(columns=["sale_date"])
predicted_prices = comparison_model.predict(X_validation)

compare_result = pd.DataFrame({"house_id": validation_data["house_id"], "price": predicted_prices})

print(compare_result)

     house_id      price
0     1331375   81722.45
1     1630115  312454.17
2     1645745  398703.76
3     1336775  107464.96
4     1888274  268045.04
..        ...        ...
295   1986255  356090.34
296   1896276  385672.71
297   1758223  259844.48
298   1752010  172590.89
299   1651404  408091.26

[300 rows x 2 columns]
